In [1]:
import logging
import pandas as pd
import numpy as np
import glob
import os
import sys
import dotenv
import requests
from datetime import datetime
#insert the location of your .env file here:
dotenv.load_dotenv('/home/eduardo/Documents/RW_github/cred/.env')
utils_path = os.path.join(os.path.abspath(os.getenv('PROCESSING_DIR')),'utils')
if utils_path not in sys.path:
    sys.path.append(utils_path)
import util_files
import util_cloud
import util_carto
import shutil
from zipfile import ZipFile



In [2]:
# Set up logging
# Get the top-level logger object
logger = logging.getLogger()
for handler in logger.handlers: logger.removeHandler(handler)
logger.setLevel(logging.INFO)
# make it print to the console.
console = logging.StreamHandler()
logger.addHandler(console)
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# name of table on Carto where you want to upload data
# using preexisting table for this dataset
dataset_name = 'soc_002_rw1_gender_development_index' #check

logger.info('Executing script for dataset: ' + dataset_name)
# create a new sub-directory within your specified dir called 'data'
# within this directory, create files to store raw and processed data
data_dir = util_files.prep_dirs(dataset_name)

Executing script for dataset: soc_002_rw1_gender_development_index


In [4]:
'''
Download data and save to your data directory
Data can be downloaded from:
http://hdr.undp.org/en/indicators/137906#
'''

# download the data from the source
logger.info('Downloading raw data')
download = glob.glob(os.path.join(os.path.expanduser("~"), 'Downloads', 'Gender Development Index (GDI).csv'))[0]

# move this file into your data directory
raw_data_file = os.path.join(data_dir, os.path.basename(download))
shutil.move(download,raw_data_file)



'data/Gender Development Index (GDI).csv'

In [5]:
# read in data to pandas dataframe
df = pd.read_csv(raw_data_file, sep = ',',header=0,  skiprows=5, encoding='latin-1')
# remove rows that doesn't contain countries information
df.drop(df.tail(18).index,inplace=True)
# remove unnamed columns
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
# replace empty spaces in column names
df.columns = df.columns.str.replace(' ', '_')
# replace ".." with np.nan
df = df.replace('..', np.nan)
# remove whitespace at the beginning and end of strings
df = df.replace(to_replace="^\s+",value = '',regex=True)
# convert the column names to lowercase
df.columns = [x.lower() for x in df.columns]
# convert tables from wide form (each year is a column) to long form
df = pd.melt(df,id_vars=['hdi_rank', 'country'],var_name='year', value_name='gdi_value')
# change dtypes of columns
#df['hdi_rank'] = df['hdi_rank'].astype('int64')
#df['gdi_value'] = df['gdi_value'].astype('float64')
#df['year'] = df['year'].astype('int64')
#df['datetime'] = [datetime(x, 1, 1) for x in df.year]
# replace all NaN with None
df=df.where((pd.notnull(df)), None)# save processed dataset to csv

# save processed dataset to csv
#processed_data_file = os.path.join(data_dir, dataset_name+'_edit.csv')
#df.to_csv(processed_data_file, index=False)

In [6]:
df.head()

,hdi_rank,country,year,gdi_value
0,169,Afghanistan,1995,None
1,69,Albania,1995,0.938
2,91,Algeria,1995,None
3,148,Angola,1995,None
4,46,Argentina,1995,0.987


In [63]:
# Extracting year from datetime column
#df['year'] = df['year'].dt.year
#df['year'] = datetime.strptime(df['year'], '%Y')
test = df['country'].tolist()
test

['Afghanistan',
 'Albania',
 'Algeria',
 'Angola',
 'Argentina',
 'Armenia',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bhutan',
 'Bolivia (Plurinational State of)',
 'Bosnia and Herzegovina',
 'Botswana',
 'Brazil',
 'Brunei Darussalam',
 'Bulgaria',
 'Burkina Faso',
 'Burundi',
 'Cabo Verde',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Central African Republic',
 'Chad',
 'Chile',
 'China',
 'Colombia',
 'Comoros',
 'Congo',
 'Congo (Democratic Republic of the)',
 'Costa Rica',
 'Croatia',
 'Cuba',
 'Cyprus',
 'Czechia',
 "Côte d'Ivoire",
 'Denmark',
 'Dominican Republic',
 'Ecuador',
 'Egypt',
 'El Salvador',
 'Estonia',
 'Eswatini (Kingdom of)',
 'Ethiopia',
 'Fiji',
 'Finland',
 'France',
 'Gabon',
 'Gambia',
 'Georgia',
 'Germany',
 'Ghana',
 'Greece',
 'Guatemala',
 'Guinea',
 'Guyana',
 'Haiti',
 'Honduras',
 'Hong Kong, China (SAR)',
 'Hungary',
 'Iceland',
 'India',
 'Indonesia',
 'Ira

In [49]:
# not part of script just to know countries that need the alias
alias = pd.read_csv('rw_aliasing_countries.csv')
alias.head()

,the_geom,cartodb_id,alias,name,iso
0,NaN,1,Korea North,North Korea,PRK
1,NaN,2,Korea South,South Korea,KOR
2,NaN,4,Congo Brazzaville,Republic of Congo,COG
3,NaN,5,UAE,United Arab Emirates,ARE
4,NaN,6,Bosnia,Bosnia and Herzegovina,BIH


In [60]:
alias_name = alias['alias'].tolist()
df_country = df['country'].tolist()
main_list = np.setdiff1d(df_country,alias_name)
main_list

array(['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia',
       'Austria', 'Azerbaijan', 'Bahrain', 'Bangladesh', 'Barbados',
       'Belarus', 'Belgium', 'Belize', 'Benin', 'Bhutan',
       'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'Bulgaria',
       'Burkina Faso', 'Burundi', 'Cambodia', 'Cameroon', 'Canada',
       'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia',
       'Comoros', 'Costa Rica', 'Croatia', 'Cuba', 'Cyprus', 'Denmark',
       'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt',
       'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia',
       'Ethiopia', 'Fiji', 'Finland', 'France', 'Gabon', 'Gambia',
       'Georgia', 'Germany', 'Ghana', 'Greece', 'Grenada', 'Guatemala',
       'Guinea', 'Guyana', 'Haiti', 'Honduras', 'Hong Kong, China (SAR)',
       'Hungary', 'Iceland', 'India', 'Indonesia', 'Iraq', 'Ireland',
       'Israel', 'Italy', 'Jamaica', 'Ja